In [10]:
import psycopg as pg3
import psycopg2 as pg
import psycopg2.extensions
from psycopg2.extras import execute_values
import copy
import time
import gzip
import json
import os
import dotenv

In [21]:
# Connect to your postgres DB
connection = pg.connect(host="localhost", user="postgres", password="example", dbname="postgres")
pg.extensions.register_type(pg.extensions.UNICODE, connection)

In [22]:
cursor = connection.cursor()


In [3]:
path_to_authors = r"C:\Users\marve\authors.jsonl.gz"
path_to_conversations = r"C:\Users\marve\conversations.jsonl.gz"

In [4]:
with gzip.open(path_to_conversations,'r') as f:
    for line in f:
        conversation = line
        break

Zacneme prvotne so importom dat userov...

In [14]:
def exists(obj, attr, is_id=False):    
    if attr in obj.keys() and obj[attr] is not None:
        if is_id and obj[attr] == "":
            return False
        return True
    return False

In [41]:
def make_string_valid(string):
    string = (str(string)
        .encode("utf-8")
        .decode("utf-8", errors="replace")
        .replace("\x00", "\uFFFD")
    )
    return string

In [19]:
arr = [
    ["a", "b"],
    ["a", "b"] 
]

In [25]:
arr = [0, 0, 1]

In [23]:
set(arr)

TypeError: unhashable type: 'list'

In [7]:
# v ramci definicie tabulky vynechavam FK contraints, nakolko zbytocne budu spomalovat tvorbu dat a skor si to budem u
# seba kontrolovat, ci existuje entita, na ktoru sa referencuje...

# aj sme odstranili unique constraint od hashtags
def create_misc_tables_no_fk(cursor):

    

    
    

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS context_domains (
        id int8 PRIMARY KEY,
        name varchar(255) NOT NULL,
        description text
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS context_entities (
        id int8 PRIMARY KEY,
        name varchar(255) NOT NULL,
        description text
        );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS context_annotations (
        id int8 PRIMARY KEY,
        conversation_id int8 NOT NULL,
        context_domain_id int8 NOT NULL,
        context_entity_id int8 NOT NULL
        );
    """)


    cursor.execute()
    
      
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS links (
        id int8 PRIMARY KEY,
        conversation_id int8 NOT NULL,
        url varchar(2048) NOT NULL,
        title text,
        description text
        );
    """)

    cursor.execute(
        """CREATE TABLE IF NOT EXISTS conversation_references (
        id int8 PRIMARY KEY,
        conversation_id int8 NOT NULL,
        parent_id int8 NOT NULL,
        type varchar(20) NOT NULL
        );
    """)
    

In [18]:
sum([True, True])

2

In [16]:
# conversation = json.loads(conversation)
conversation

{'author_id': '91305838',
 'conversation_id': '1496732914436001794',
 'created_at': '2022-02-24T06:25:09.000Z',
 'entities': {'mentions': [{'end': 19,
    'id': '1708603484',
    'start': 3,
    'username': 'aliilkayozdemir'}]},
 'id': '1496732914436001794',
 'lang': 'tr',
 'possibly_sensitive': False,
 'public_metrics': {'like_count': 0,
  'quote_count': 0,
  'reply_count': 0,
  'retweet_count': 20},
 'referenced_tweets': [{'id': '1496731235300917250', 'type': 'retweeted'}],
 'reply_settings': 'everyone',
 'source': 'Twitter Web App',
 'text': 'RT @aliilkayozdemir: Batı istihbaratı "Katil Rusya\'nın" işgal planını açıkladıkça, "Batı, Rusya\'yı savaşa sokmak istiyor" diye günlerdir, i…'}

In [8]:
def import_context_tables(path_to_conversations_export, log_step=10000):
    pass

In [16]:
def import_author_table(path_to_author_export, log_step=10000):
    print("...Importing 'authors' data...")
    start_time = time.time()
    prev_block_time = start_time

    create_table_string = """
        CREATE TABLE IF NOT EXISTS authors (
	    id int8 PRIMARY KEY,
        name varchar(255),
        username varchar(255),
        description text,
        followers_count int4,
        following_count int4,
        tweet_count int4,
        listed_count int4
        );
    """

    connection = pg.connect(host="localhost", user="postgres", password="example", dbname="postgres")
    pg.extensions.register_type(pg.extensions.UNICODE, connection)
    cursor = connection.cursor()

    cursor.execute(create_table_string)

    with gzip.open(path_to_author_export,'r') as f:
        for it, author_json_str in enumerate(f):
            author_obj = json.loads(author_json_str)
            author_row = migrate_author_entity(author_obj)
            if author_row is not None:
                cursor.execute(
                    """
                    INSERT INTO authors (id, name, username, description, 
                    followers_count, following_count, tweet_count, 
                    listed_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """,
                    author_row
                )

            if it % 10000 == 0 and it != 0:
                connection.commit()

                time_check = time.time()
                print(f"Time elapsed since the beggining: {time_check - start_time / 60}m | Time spent on the last block: {time_check - prev_block_time}")
                prev_block_time = time_check

            
    cursor.close()
    connection.close()
    print("...Finished importing 'authors' data...")    
        

In [36]:
def migrate_author_entity(original_obj):
    obj = copy.deepcopy(original_obj)

    if exists(obj, "id", is_id=True) == False:
        return None
    obj["id"] = int(obj["id"])

    nullable_string_attributes = [
        "name",
        "username",
        "description"
    ]
    for str_attrb in nullable_string_attributes:
        if exists(obj, str_attrb) == False:
            obj[str_attrb] = None
        else:
            obj[str_attrb] = make_string_valid(obj[str_attrb])
    
    public_metrics = [
        "followers_count",
        "following_count"
        "tweet_count"
        "listed_count"
    ]
    if exists(obj, "public_metrics") == False:
        obj["public_metrics"] = {}
        
        for m in public_metrics:
            obj["public_metrics"][m] = None
    else:    
        for m in public_metrics:
            if exists(obj["public_metrics"], m) == False:
                obj["public_metrics"][m] = None
            else:
                obj["public_metrics"][m] = int(obj["public_metrics"][m])
    
    table_row = [
        obj["id"],
        obj["name"],
        obj["username"],
        obj["description"],
        obj["public_metrics"]["followers_count"],
        obj["public_metrics"]["following_count"],
        obj["public_metrics"]["tweet_count"],
        obj["public_metrics"]["listed_count"],
    ]
    

    return table_row

In [30]:
author_row = migrate_author_entity(author)

In [32]:
cursor = connection.cursor()

cursor.execute(
    """
    INSERT INTO authors (id, name, username, description, 
    followers_count, following_count, tweet_count, 
    listed_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """,
    author_row
)

In [33]:
connection.commit()